# **Wall-E Bot**

### *19/02/2023*

This is my first personal project since my master's thesis. I have decided to create a personal chat bot based on my whatsapp chat history. Though I thought creating such bot would be super difficult, after some research and learning I learned that it is very easy to create a basic bot which can provide funny responses. Python has a wonderful library called [chatterbot](https://chatterbot.readthedocs.io/en/stable/) which offers a complete package to create such bot. Since this library is not language based the chat language doesn't matter. As I am bilingual, most of my chats involve both English and [Tamil](https://en.wikipedia.org/wiki/Tamil_language) words.

In [1]:
pip install chatterbot==1.0.4 pytz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.2/526.2 KB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 KB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 KB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 87.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.5/269.5 KB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.6/270.6 KB 35.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.2.19-cp38-cp38-linux_x86_64.whl size=1160260 sha256=321a51048abdedf6b70a1f75732349b926b8b2bd233106da77b39230e9696f21
  Stored in directory: /root/.cache/pip/wheels/0d/13/f8/47c2f3157957c3693caffa64a94a

Follow the [instructions](https://faq.whatsapp.com/1180414079177245/?cms_platform=android) from Whatsapp's official FAQ to export the chat history. Export it without media attachments. 

This text file has some metadata which isn't required for our chatbot. 

Whatsapp chat raw data needs some cleaning to be done. Below code extracts the data from the text file and removes the metadata, later removing it concats the chat when two or more consecutive chat lines belong to the same individual. 

Example:

This chat:

22/08/2022, 17:37 - Me: Hai

22/08/2022, 17:41 - Me: 😀😀😀

22/08/2022, 17:45 - Me: What are you doing?

22/08/2022, 17:45 - You: Working

22/08/2022, 17:46 - You: 🥺🥺🥺🥺🥺🥺

22/08/2022, 19:28 - Me: Ok

Transfers to a tuple:

('Hai 😀😀😀 What are you doing?','Working 🥺🥺🥺🥺🥺🥺','Ok')

Store the chat history file as "chat.txt" in the storage.


In [2]:
import pandas as pd
df = pd.read_csv('chat.txt', sep='漢ழ்字', engine='python',header=None)
df['Date'] = df[0].str[0:20]
df['Text'] = df[0].str[20:]
df = df.drop(columns = [0,'Date'])
df['Sender'] = df['Text'].str[0:3]
df['Text'] = df['Text'].str[3:]
dp = pd.DataFrame(columns = ["Sender", "Text"])

for index,row in df.iterrows():
    if dp.empty:
        dp = pd.concat([dp, pd.DataFrame.from_records([{'Sender':df['Sender'][index],'Text':df['Text'][index]}])])
    elif df['Sender'][index]!=dp['Sender'].iloc[-1]:
        dp = pd.concat([dp, pd.DataFrame.from_records([{'Sender':df['Sender'][index],'Text':df['Text'][index]}])])
    elif df['Sender'][index]==dp['Sender'].iloc[-1]:
        dp['Text'].iloc[-1] = dp['Text'].iloc[-1] +" "+df['Text'][index]


In [5]:
open_req_list = tuple(list(dp['Text']))

Now its time to train the model with the dataset. 

In [7]:
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

chatbot = ChatBot("WallE")

trainer = ListTrainer(chatbot)
trainer.train(open_req_list)

exit_conditions = (":q", "quit", "exit")
while True:
    query = input("> ")
    if query in exit_conditions:
        break
    else:
        print(f"🪴 {chatbot.get_response(query)}") 

List Trainer: [                    ] 0%

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


List Trainer: [####################] 100%
> Hi what are you doing?
🪴 Okie Happy pungal
> thanks
🪴 😘
> what are you doing?
🪴 Working
> too much work?
🪴 Good 🤣🤣
> hmm
🪴 She asked as long i has the vaccine on our after 2012 is cool
> :q
